In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Data

In [2]:
df = pd.read_csv('data_20180119_20180420.csv', index_col=0)
df.head()

,t0,T1,K1,C1,T2,K2,C2,S0,Adj_S0
0,2018-01-02,2018-01-19,1000.0,191.525,2018-04-20,800.0,396.725000,59.4505,1189.01001
1,2018-01-02,2018-01-19,1100.0,92.525,2018-04-20,1015.0,191.675000,59.4505,1189.01001
2,2018-01-02,2018-01-19,1150.0,46.075,2018-04-20,1025.0,183.025000,59.4505,1189.01001
3,2018-01-02,2018-01-19,1160.0,38.000,2018-04-20,1050.0,157.776341,59.4505,1189.01001
4,2018-01-02,2018-01-19,1170.0,30.450,2018-04-20,1080.0,137.850000,59.4505,1189.01001


In [4]:
t0List = list(df.t0.unique())

# dimension: K[k-th t0][t1 and t2][i-th option with i-th strike]
K = [[np.array(df.loc[df.t0 == t0,'K1']),np.array(df.loc[df.t0==t0,'K2'])] for t0 in t0List]

# dimension: Pi[k-th t0][t1 and t2][i-th option with i-th price]
Pi = [[np.array(df.loc[df.t0 == t0,'C1']),np.array(df.loc[df.t0==t0,'C2'])] for t0 in t0List]

In [5]:
# number of t0
N_t0 = len(t0List)

# number of times (t1, t2)
T = 2

# number of options in hedging
N1, N2 = 20, 20
Size = max(N1, N2)

In [6]:
# Assume marginal distribution is uniform
def margDistr(batch_size):
    data = np.random.uniform(size=(batch_size, T))
    for t in range(T):
        data[:, t] = S_min + (S_max - S_min) * data[:, t] # just a simple test, need further consideration
    return data

In [8]:
# arbitrary
S_t0 = 1189
S_max = 1600
S_min = 800
BATCH_SIZE = 1000

S = margDistr(BATCH_SIZE)

In [9]:
Phi = (S[:,1]-S[:,0])
# define the payoff function of the derivative

### Variables

In [10]:
# parameter: d
d = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, 1], dtype=tf.float64),
    trainable = True,
    name = 'd'
)

In [11]:
# parameter: lambda
Lambda = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, T, Size], dtype=tf.float64),
    trainable = True,
    name = 'lambda'
)

In [ ]:
# # parameter: Delta
# Delta = tf.Variable(
#     initial_value = tf.random.normal(shape=[N_t0, T-1], dtype=tf.float64),
#     trainable = True,
#     name = 'Delta'
# )

In [14]:
# neural network: Delta(S_1)
Delta = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(BATCH_SIZE,)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(N_t0, activation='tanh', dtype=tf.float64)
])

### The step-by-step computation of the objective function

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2}$

In [15]:
hedgePrice = []
for n in range(N_t0):
    p = d[n]
    for t in range(T):
        p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
    hedgePrice.append(p)

In [16]:
# take a look
hedgePrice

[<tf.Tensor: shape=(1,), dtype=float64, numpy=array([-63.66629384])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-467.91346297])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-1133.04774071])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-385.49222492])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-137.59902128])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([867.22868106])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([902.01608627])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-1128.92346692])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-967.99200335])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([157.80523112])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([250.146957])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([363.99873157])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([25.98719897])>]

$d+\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^++\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^++\Delta_{1}(S_1)\cdot(S_2-S_1)$

In [17]:
hedgeTerm = []
for n in range(N_t0):
    p = d[n]
    for t in range(T):
        p += tf.reduce_sum(Lambda[n,t,:]*np.maximum(S[:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
#     for t in range(T-1):
#         p += Delta[n,t]*(S[:,t+1]-S[:,t])
    p += Delta(S[:,0].reshape((1,-1)))[0,n]*(S[:,1]-S[:,0])
    hedgeTerm.append(p)

In [18]:
# take a look
hedgeTerm

[<tf.Tensor: shape=(1000,), dtype=float64, numpy=
 array([-9.99184138e+02,  7.33621149e+01, -6.82711424e+02, -1.86372311e+02,
         3.05502194e+02, -2.95416577e+02, -9.25104502e+02, -6.86866158e+02,
        -2.53222943e+02, -9.33179089e+02, -1.33826705e+03, -3.96921504e+02,
        -1.41065193e+03,  6.32053873e+01, -1.69674506e+02, -3.11892526e+02,
        -1.15760490e+03, -1.14775616e+03,  1.05787813e+02,  9.81548996e+01,
        -1.06235678e+03,  1.03852355e+02, -7.82604033e+02, -6.82543070e+02,
        -2.73207035e+02,  1.05268863e+02, -7.97219187e+02, -2.61345552e+02,
        -3.13837609e+02, -1.04489152e+02,  7.77285684e+01,  2.08799654e+02,
         7.66850815e+01,  1.72024025e+02,  1.72612509e+02,  2.24643162e+02,
         2.22139961e+02, -5.71878512e+02,  4.56953039e+01, -7.62149263e+02,
        -9.29350104e+02, -1.15235410e+03,  2.40963821e+02, -9.19834309e+02,
         3.33514152e+01,  3.42842362e+02,  2.01490730e+02, -2.34001314e+02,
        -3.88515739e+02,  1.04730189e+

$\inf_{h\in\mathcal{H}^m}\int hd\mu_0+\int \beta_\gamma(c-h)d\theta$

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2} + \Gamma\cdot\left[\left(\Phi(S_1,S_2)-d-\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^+-\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^+-\Delta_{1}(S_1)\cdot(S_2-S_1)\right)^+\right]^2$

In [19]:
# objective function
Price = tf.reduce_mean(hedgePrice, axis=1)

Gamma = 100
diff = Phi - tf.stack(hedgeTerm, axis=0)
Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)

ObjFunc = Price + Gamma*Penalty

In [20]:
# take a look
Price

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([  -63.66629384,  -467.91346297, -1133.04774071,  -385.49222492,
        -137.59902128,   867.22868106,   902.01608627, -1128.92346692,
        -967.99200335,   157.80523112,   250.146957  ,   363.99873157,
          25.98719897])>

In [21]:
# take a look
Penalty

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([5.88888628e+05, 9.02725573e+05, 1.23960820e+06, 6.66004735e+05,
       1.38697404e+05, 7.67689837e-01, 1.31345706e+02, 3.24451792e+06,
       2.71726023e+06, 1.40615509e+05, 1.09812641e+05, 6.13352300e+04,
       2.06379208e+05])>

In [22]:
# take a look
ObjFunc

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([5.88887991e+07, 9.02720894e+07, 1.23959687e+08, 6.66000880e+07,
       1.38696028e+07, 9.43997665e+02, 1.40365867e+04, 3.24450663e+08,
       2.71725055e+08, 1.40617087e+07, 1.09815143e+07, 6.13388699e+06,
       2.06379468e+07])>

### Optimisation

In [23]:
# Combine all the computation as a function
def ObjFunc(d, Lambda, Delta, K, Pi, Phi, S):
    hedgePrice = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
        hedgePrice.append(p)
    
    hedgeTerm = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*np.maximum(S[:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
#         for t in range(T-1):
#             p += Delta[n,t]*(S[:,t+1]-S[:,t])
        p += Delta(S[:,0].reshape((1,-1)))[0,n]*(S[:,1]-S[:,0]) # Delta as a neural network
        hedgeTerm.append(p)
    
    Price = tf.reduce_mean(hedgePrice, axis=1)
    Gamma = 100
    diff = Phi-tf.stack(hedgeTerm, axis=0)
    Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)
    
    return Price + Gamma*Penalty, Price

In [24]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = 0.01)

In [101]:
# for step in range(10000):
#     with tf.GradientTape() as tape:
#         obj, price = ObjFunc(d, Lambda, Delta, K, Pi, Phi, S)
#     gradients = tape.gradient(obj, [d, Lambda, Delta])
#     optimizer.apply_gradients(zip(gradients, [d, Lambda, Delta]))

In [25]:
for step in range(10000):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(d)
        tape.watch(Lambda)
        for variable in Delta.trainable_variables:
            tape.watch(variable)

        obj, price = ObjFunc(d, Lambda, Delta, K, Pi, Phi, S)
    
    gradients_d, gradients_Lambda = tape.gradient(obj, [d, Lambda])
    gradients_Delta = tape.gradient(obj, Delta.trainable_variables)
    optimizer.apply_gradients(
        zip([gradients_d, gradients_Lambda] + gradients_Delta, [d, Lambda] + Delta.trainable_variables)
    )
    
    del tape

In [26]:
price

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([  510.10419517,   -13.64604934,   733.71221072,  3011.38562227,
          40.23180068,  -375.1906295 ,  -143.4686607 ,  2585.61185331,
        2443.99003766,  2687.19425759,  2846.9881979 , -1016.69570141,
        2880.79313663])>

In [27]:
obj

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([ 5.13293114e+02, -1.27856669e+01,  7.41917839e+02,  8.21415828e+04,
        4.14354923e+01, -3.74781145e+02, -1.42904911e+02,  6.11617867e+04,
        1.00390976e+05,  2.60640673e+05,  2.62811319e+05, -1.01635210e+03,
        2.71378199e+05])>